In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

## 1. URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)
df_a=pd.DataFrame(alle_seiten)
# Die Geschäftsnummer zur Zahl machen. 
df_a["VorlageNR"] = df_a["VorlageNR"].astype(int) 

## 1.1 Selektion: Fokus auf Vorstösse
So will ich eine allzu grosse pdf-Schwemme verhindern.
Die Idee: nur jene Geschäfte auf den PC ziehen, die auch relevant sind für die Auswertung. 
Also nur Vorstösse. Mich so bis zu den PDF's vorkämpfen, um auch hier nur jene auf meinem Rechner zu haben, 
die ich wirklich auf meinem Rechner will.

In [ ]:
#hier mache ich eine Liste, die nur aus denjenigen URLs besteht, die Vorstösse sind. 

list_of_vorstoss=["Motion","Interpellation","Postulat","Kleine Anfrage", "Initiative"]
df_v=df_a[df_a["Art des Geschäfts"].isin (list_of_vorstoss)]

In [ ]:
df_v #hier habe ich nun alle Vorstösse

In [ ]:
df_v=df_v.reset_index()#Ich will überall nun denselben Index. deshalb nur soviel, wie ich Zeilen habe
df_v.to_csv("KRFrames/df_v_Hauptseiten")

### 1.1.1 Eine Liste aus allen Geschäfts-URLs machen (um später auf diese zuzugreiffen)

In [ ]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_v["url"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

In [ ]:
len(url_liste_ganz)

### Die einzelnen Geschäfte aufrufen und Infos rausholen

# -> Einschub: ich will speichern und mit diesen Daten dann arbeiten.

In [ ]:
#habe nun alle Seiten der Geschäfte auf meinem Rechner gespeichert. 
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    #Es sind nun exakt so viele Seiten auf meinem Rechner, wie das DataFrame oben Zeilen anzeigt

# <-Einschub-Ende. Ich habe es geschafft, das zu speichern. 

In [ ]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]#####ACHTUNG ich mache den RANGE kleiner, um weniger GEschäfte zu erhalten.

for seite_g in range(0,1220): #Range für alle Geschäfte wäre 1219!
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [ ]:
len(geschaefts_liste)

In [ ]:
geschaefts_liste[-10:]

# 2. Zugang zu den PDFs

Ich musste rausfinden, bei welchen Files ich nicht auf die Nummer 1 zugreiffen kann. 
*Siehe File: "Falsche Files rausfinden - code anpassen"* 

In [ ]:
#hier  habe ich mich entschieden, auf den Zugriff der Namen zu verzichten, und dies einheitlich über die PDFs zu lösen.
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
eingang_list=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
   
   
     #Da es zum Teil Referenzen hat und dadurch ein anderer Ort abgegriffen werden muss, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    elif td_g_list[-1].find_all("a")[0].text== "2": #bei älteren GEschäften sind es Bericht&Anträge
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else: 
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
   
            
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [ ]:
df_g=pd.DataFrame(eingang_list)

In [ ]:
#Das Datum muss noch als solches lesbar gemacht werden
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum
df_g["VorlageNR"] =df_g["VorlageNR"].astype(int)

In [ ]:
df_g.to_csv("KRFrames/df_g_LinkszuPDF")

In [ ]:
df_g=pd.read_csv("KRFrames/df_g_LinkszuPDF")

# 3. PDFs der Vorstösse runterladen

In [ ]:
for link in tqdm(df_g["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+name, "wb") as f:
        f.write(r.content)

# 4. PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [ ]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [ ]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [ ]:
len(pdf_files_list) #Ich habe ja die PDFs ignoriert, zu welchen ich auf Grund der Referenz keine PDF-Links hatte.

In [ ]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr, "Filename":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [ ]:
df_p=pd.DataFrame(dat_pdf_list)

In [ ]:
df_p.sort_values("Einreichedatum") #Stichproben haben gezeigt, dass die fehlerhaften Datumsangaben in GEschäftern sind,
# Welche selber meist schon ein Datum haben. 

In [ ]:
df_p.to_csv("KRFrames/INFOSausPDF")

## 4.1 Zeit in Datetime umwandeln
schwierigkeit: wie bringe ich das deutsche Zeitformat in englisches. Mache ich es falsch oder gibts wieder probleme mit den Grossbuchstaben der Zeitformate? Hatten wir windowsuser auch schon.

In [ ]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE') # hier die Sprachwahl
df_p["Einreichedatum"] = pd.to_datetime(df_p["Einreichedatum"], format= "%d. %B %Y", errors='coerce') #Angabe des Formats

In [ ]:
 df_p #Halleluia:-) habs doch noch geschafft. Und die wenigen, die falsch formiert sind, werden nicht auffallen.

In [ ]:
df_p.to_csv("KRFrames/KonvertierteDatenformate")

In [ ]:
df_p=pd.read_csv("KRFrames/KonvertierteDatenformate")

In [ ]:
#Nun noch ohne.0 hinten bei der VorlageNR. Ah es ist ein Float. Kann es also zu int oder str machen
df_p["VorlageNR"]=df_p["VorlageNR"].astype(str)

In [ ]:
def splitting(elem):
    elem = elem.split('.')[0].strip()
    return elem


In [ ]:
df_p["VorlageNR"]=df_p["VorlageNR"].apply(splitting) # So, nun sind die .0 weg. Mal schauen. Das String-Format reicht wohl
del df_p["Unnamed: 0"] # die beiden Spalten entfernen
del df_p["Unnamed: 0.1"]
df_ps=df_p.rename(columns={"Filename": "Link Vorstoss-PDF"}) #Spalte gleich benennen wie im anderen Frame

In [ ]:
df_ps.to_csv("KRFrames/df_p_Datum") # und speichern

## 4.2 Ein Dataframe machen, mit den PDF-Dokumentennamen
- splitting funktion schreiben
- auf Spalte anwenden
- neu abspeichern 

In [ ]:
def splittpdf(elem):
    elem = elem.split('/')[-1].strip()
    return elem

In [ ]:
df_g["Link Vorstoss-PDF"]=df_g["Link Vorstoss-PDF"].apply(splittpdf)

In [ ]:
df_g.to_csv("KRFrames/df_g_Datum_mit_PDF")

## 4.3 Nun zusammenfügen der beiden Daten-Dataframes anhand der PDF-Namen

In [ ]:
df_g1=pd.read_csv("KRFrames/df_g_Datum_mit_PDF")
df_p1=pd.read_csv("KRFrames/df_p_Datum")

In [ ]:
df_dat_max=pd.merge(df_g1, df_p1, how="left",on= "Link Vorstoss-PDF")

In [ ]:
df_dat_max.sort_values("Einreichedatum_x").tail(100)  #So, jetzt sind zwar beide zusammen, aber noch nicht vereint
#Jetzt noch irgendwie die beiden Einreichedaten miteinander ableichen/ergängen.

### 4.3.1 die Datenspalten zusammenbekommen.

In [ ]:
#hier füge ich sie zusammen
df_dat_zus=df_dat_max.Einreichedatum_x.combine_first(df_dat_max.Einreichedatum_y)

In [ ]:
df_dat_max['Einreichedatum'] = df_dat_zus # hier füge ich die neu enstandene "Series" dem Dataframe an

In [ ]:
df_datum=df_dat_max[["Unnamed: 0_x","VorlageNR_x", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_datum.to_csv("KRFrames/df_datum_ALLE")

# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [2]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Alle_Namen_mit_Allianz_und_ohne")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



In [3]:
#  zuerst vorputzen
df_v1 = df_v[["VorlageNR", "Geschäft"]].copy()
df_v1

,VorlageNR,Geschäft
0,3020,Postulat von Luzian Franzini und Esther Haas b...
1,3019,"Motion von Luzian Franzini, Rita Hofer, Tabea ..."
2,3018,"Interpellation von Fabio Iten, Laura Dittli un..."
3,3017,"Postulat von Tabea Zimmermann Gibson, Stéphani..."
4,3014,Interpellation der SVP-Fraktion betreffend mob...
...,...,...
1215,762,Motion der erweiterten Justizprüfungskommissio...
1216,666,Motion der Kommission Teilrevision Personalges...
1217,304,Motion von Manuela Weichelt betreffend HIV-Prä...
1218,282,Motion von Christoph Hohler betreffend Radstre...


In [4]:
df_dat_all=df_dat_all.rename(columns={"VorlageNR_x": "VorlageNR"})

In [5]:
df_dat_all

,Unnamed: 0,Unnamed: 0_x,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,0,0,3020,2019-10-11,3020-1-16168_Racial-Profiling.pdf
1,1,1,3019,2019-10-11,3019-1-16167_Chancengleichheit.pdf
2,2,2,3018,2019-10-07,3018-1-16166_Praktikum.pdf
3,3,3,3017,2019-10-06,3017-1-16165_Frauenwahl-stimmrecht_Feier.pdf
4,4,4,3014,2019-09-26,3014-1-16159_Geschwindigkeitskontrollen-2-0.pdf
...,...,...,...,...,...
1215,1215,1215,762,2005-10-31,pdoc_1047_1.pdf
1216,1216,1216,666,2002-06-06,pdoc_55_1.pdf
1217,1217,1217,304,2003-05-27,pdoc_50_1.pdf
1218,1218,1218,2635,1995-08-07,1620


In [6]:
# Nun zusammenfügen
df_alles=pd.merge(df_v1, df_dat_all, how="inner",on= "VorlageNR")

In [7]:
df_alles

,VorlageNR,Geschäft,Unnamed: 0,Unnamed: 0_x,Einreichedatum,Link Vorstoss-PDF
0,3020,Postulat von Luzian Franzini und Esther Haas b...,0,0,2019-10-11,3020-1-16168_Racial-Profiling.pdf
1,3019,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",1,1,2019-10-11,3019-1-16167_Chancengleichheit.pdf
2,3018,"Interpellation von Fabio Iten, Laura Dittli un...",2,2,2019-10-07,3018-1-16166_Praktikum.pdf
3,3017,"Postulat von Tabea Zimmermann Gibson, Stéphani...",3,3,2019-10-06,3017-1-16165_Frauenwahl-stimmrecht_Feier.pdf
4,3014,Interpellation der SVP-Fraktion betreffend mob...,4,4,2019-09-26,3014-1-16159_Geschwindigkeitskontrollen-2-0.pdf
...,...,...,...,...,...,...
1140,801,Motion von Hans Abicht betreffend Raumkonzept ...,1214,1214,2006-08-22,pdoc_1440_1.pdf
1141,762,Motion der erweiterten Justizprüfungskommissio...,1215,1215,2005-10-31,pdoc_1047_1.pdf
1142,666,Motion der Kommission Teilrevision Personalges...,1216,1216,2002-06-06,pdoc_55_1.pdf
1143,304,Motion von Manuela Weichelt betreffend HIV-Prä...,1217,1217,2003-05-27,pdoc_50_1.pdf


In [9]:
df_np

,Unnamed: 0,Namenganz,P
0,0,Judith Wild,FDP
1,1,Madeleine Landolt,SGA
2,2,Peter Bossard,FDP
3,3,Susi Frei,FDP
4,4,Alois Henggeler,CVP
...,...,...,...
386,312,Iris Hess-Brauer,CVP
387,334,Mariann Hess-Witschi,Kt.
388,343,Magda Feldmann-Müller,CSP
389,345,Sepp Grob-Bieri,CVP


In [8]:
df_alles_ganz=pd.merge(df_alles, df_np, how="inner", on="VorlageNR")
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Stand", "Zuständigkeit", "Art des Geschäfts","Status", "Link Vorstoss-PDF"]]

KeyError: 'VorlageNR'

In [ ]:
df_tuti #Wieso wird es nicht ergänzt. Der Fehler ist im File 02 Namen passiert. dort nachschauen.

# XX Zum Resultat hier mal das Dataframe der Hauptseite sowie der Unterseiten zusammenfügen.

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [ ]:
#nun will ich noch das Datum zum Idex machen
df_mit_dat.set_index("Einreichedatum", inplace=True)

In [ ]:
df_mit_dat.dtypes

# Ein erstes Ergebnis = ab 2013 alle  Vorstösse
# ACHTUNG das stimmt ja so gar nicht, da bei mehreren Personen
# auch mehrere Einreichungen verzeichnet werden!!!

In [ ]:
#Nun will ich ein erstes Mal in die DAten schauen, zumindest ab 2013 - ob es ein Muster beim Einreichen gibt.
#DAzu krame ich jetzt alle Vorstösse raus

df_vo_all=df_mit_dat[{"Art des Geschäfts" : ["Motion","Interpellation","Postulat","Kleine Anfrage"] }]

In [ ]:
df_mit_dat["Art des Geschäfts"]["GeschäftsNR"]

In [ ]:
df_vo_all["2013":].resample("SMS").count().plot(figsize=(8,4))

# Regex Baby! 
### Ich versuche mit Regex in Pandas an die Namen zu kommen

In [ ]:
#nun versuche ich mit Regex an die einzelnen Namen zu kommen.
regex_n=r"\bvon(.*und.\w*.\w*\b)"
regex_n2= r"^(\w*.\w*\b)\,.(\w*.\w*\b)"

In [ ]:
df_n1=df_a['Geschäft'].str.extract(regex_n)

In [ ]:
df_n1